In [1]:
import pandas as pd
import os
import google.oauth2.credentials
import google_auth_oauthlib.flow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from google_auth_oauthlib.flow import InstalledAppFlow

In [2]:
CLIENT_SECRETS_FILE = "client_secret.json"
SCOPES = ['https://www.googleapis.com/auth/youtube.force-ssl']
API_SERVICE_NAME = 'youtube'
API_VERSION = 'v3'
def get_authenticated_service():
    flow = InstalledAppFlow.from_client_secrets_file(CLIENT_SECRETS_FILE, SCOPES)
    credentials = flow.run_console()
    return build(API_SERVICE_NAME, API_VERSION, credentials = credentials)
# Remove keyword arguments that are not set
def remove_empty_kwargs(**kwargs):
    good_kwargs = {}
    if kwargs is not None:
        for key, value in kwargs.items():
            if value:
                good_kwargs[key] = value
    return good_kwargs

In [3]:
client = get_authenticated_service()

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=1092637132049-9c3hrod2j6b7ev9scpsdql16nn4u90tv.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fyoutube.force-ssl&state=OmmvTKlGwmfePNksx1MzJGxvOjI5aL&prompt=consent&access_type=offline
Enter the authorization code: 4/1AY0e-g7toJfgE4D6q0yrCwxrGaLs-2mrtBBe4OGM2HqF4YRXPpchsqLYJP0


In [4]:
def youtube_keyword(client, **kwargs):    
    kwargs = remove_empty_kwargs(**kwargs)
    response = client.search().list(
        **kwargs
        ).execute()    
    return response

In [5]:
def youtube_keyword(client, **kwargs):    
    kwargs = remove_empty_kwargs(**kwargs)
    response = client.search().list(
        **kwargs
        ).execute()    
    return response

(1000, 4)

In [6]:
def youtube_search (criteria,max_res):   
    #create lists and empty dataframe
    titles = []
    videoIds = []
    channelIds = []
    resp_df = pd.DataFrame()
    
    while len(titles) < max_res:
        token = None
        response = youtube_keyword(client,
                        part='id,snippet',
                        maxResults=50,
                        q=criteria,
                        videoCaption='closedCaption',
                        type='video', 
                        videoDuration='long',
                        pageToken=token) 
                                         
        for item in response['items']:        
            titles.append(item['snippet']['title'])
            channelIds.append(item['snippet']['channelTitle'])
            videoIds.append(item['id']['videoId'])
        
        token = response['nextPageToken']
        
    resp_df['title'] = titles
    resp_df['channelId'] = channelIds
    resp_df['videoId'] = videoIds
    resp_df['subject'] = criteria
    
    return resp_df

In [9]:
MyStory = youtube_search('[my+story+animated]',1000)
MyStory.shape

MyStory.head()

,title,channelId,videoId,subject
0,The Worst Animated Stories On Youtube (My Stor...,Jarvis Johnson,384qFPXCuO4,[my+story+animated]
1,These Fake Animated Stories Have Gone Too Far ...,Jarvis Johnson,VzsSLpAgNcQ,[my+story+animated]
2,REACTING TO TRUE STORY SCARY ANIMATIONS PART 1...,Marshmello Family,Ytw62uoo-EM,[my+story+animated]
3,Fake Animated Stories Are Getting Worse (Actua...,Jarvis Johnson,KWqaUHFhE8M,[my+story+animated]
4,REACTING TO REAL LIFE SCARY ANIMATIONS! TRUE S...,Bijuu Mike,D4atywMmUVc,[my+story+animated]


In [7]:
def playlist_items_list_by_playlist_id(client, **kwargs):
    # See full sample for function
    kwargs = remove_empty_kwargs(**kwargs)
    response = client.playlistItems().list(**kwargs).execute()
    return response

def get_vid_ids (play_lists):
    titles = []
    descriptions = []
    channelids = []
    vidids = []
    playlist_ids = []
    video_df = pd.DataFrame()
    for play_list in play_lists:
        #request playlist items
        state=True 
        pl_data = playlist_items_list_by_playlist_id(client,
                            part='snippet,contentDetails',
                            maxResults=50,
                            playlistId=play_list)
        token=pl_data['nextPageToken']
        
        #extract information about each video in the playlist
        
        for item in pl_data['items']:
                titles.append(item['snippet']['title'])
                descriptions.append(item['snippet']['description'])
                channelids.append(item['snippet']['channelTitle'])
                vidids.append(item['snippet']['resourceId']['videoId'])
                playlist_ids.append(item['snippet']['playlistId'])
        
        
        while state:
            try:
                pl_data = playlist_items_list_by_playlist_id(client,
                            part='snippet,contentDetails',
                            maxResults=50,
                            playlistId=play_list,pageToken=token)
                token=pl_data['nextPageToken']
                
                for item in pl_data['items']:
                    titles.append(item['snippet']['title'])
                    descriptions.append(item['snippet']['description'])
                    channelids.append(item['snippet']['channelTitle'])
                    vidids.append(item['snippet']['resourceId']['videoId'])
                    playlist_ids.append(item['snippet']['playlistId'])
                
                
            except:
                state = False 
                
            
                
    video_df['title'] = titles
    video_df['description'] = descriptions
    video_df['channelid'] = channelids
    video_df['videoids'] = vidids
    video_df['playlist_id'] = playlist_ids
            
                
    return video_df

Aqui playlist_df lee un cvs con la siguiente estructura, la primer fila la descripcion de la columna
que debe ser PlaylistID y LectureName

luego en la primer columna va el identificador de una lista de reproducion y en la segunda columna el nombre que queramos para esa lista 

In [8]:
playlist_df = pd.read_csv('listas.csv')
playlist_df.head()

,PlaylistID,LectureName
0,PLb62x91woEVOzuZjzhsJTsU9fDV3fO3O8,MyStory
1,PLPWyY8_ho2J6LDG9spxhS3f-_xj5Nfpz6,Dear Diary
2,PL-LA_5Hiw7wJtPQSsuk9CFDVvagx8tWst,Short
3,PL-LA_5Hiw7wJtPQSsuk9CFDVvagx8tWst,otra
4,PLVtWx-uH8EaqZEYNV0e9jmoX0sfjhV8VP,reddit


In [9]:
video_df = get_vid_ids(playlist_df.PlaylistID)
video_df.shape

#check results
video_df

,title,description,channelid,videoids,playlist_id
0,I Regret Cheating On My Boyfriend But He Wont ...,★ DO YOU WANT TO GET YOUR STORY ANIMATED And F...,My Story Animated,m_xT1dmGEA4,PLb62x91woEVOzuZjzhsJTsU9fDV3fO3O8
1,I Stopped Eating For 2 Weeks To Get Thinner,★ DO YOU WANT TO GET YOUR STORY ANIMATED And F...,My Story Animated,znvv0qk7a-A,PLb62x91woEVOzuZjzhsJTsU9fDV3fO3O8
2,I Cant Stop Stealing From My Friends And Family,★ DO YOU WANT TO GET YOUR STORY ANIMATED And F...,My Story Animated,AkDbnUu0uw4,PLb62x91woEVOzuZjzhsJTsU9fDV3fO3O8
3,I Ran Away From Home It Was Terrifying,★ DO YOU WANT TO GET YOUR STORY ANIMATED And F...,My Story Animated,4OdmJPGG7f0,PLb62x91woEVOzuZjzhsJTsU9fDV3fO3O8
4,My Heart Is Broken Forever Because Of My Mother,I Hated My Mother | My heart is broken forever...,My Story Animated,VjrzlEt_uuU,PLb62x91woEVOzuZjzhsJTsU9fDV3fO3O8
...,...,...,...,...,...
645,"Crime Scene Cleanup Crew Members, What's Your ...","► Crime Scene Cleanup Crew Members, What's You...",Reddit Reveals,ymRMOwLYATM,PLVtWx-uH8EaqZEYNV0e9jmoX0sfjhV8VP
646,What Happens In Real Life S*x That You Never S...,► What Happens In Real Life S*x That You Never...,Reddit Reveals,n39Af7Rt8-4,PLVtWx-uH8EaqZEYNV0e9jmoX0sfjhV8VP
647,Teachers Reveal Their Most NSFW Story From Sch...,► Teachers Reveal Their Most NSFW Story From S...,Reddit Reveals,HgJtIHp2ySk,PLVtWx-uH8EaqZEYNV0e9jmoX0sfjhV8VP
648,What Are Some “Girl Secrets” Guys Don’t Know A...,► What Are Some “Girl Secrets” Guys Don’t Know...,Reddit Reveals,VjZSuG45fn0,PLVtWx-uH8EaqZEYNV0e9jmoX0sfjhV8VP


In [8]:
video_df.to_excel('All.xlsx',index=False)

In [10]:
video_df.drop(video_df[video_df['title']=='Private video'].index,inplace=True)  

video_df = video_df.reset_index()

video_df



,index,title,description,channelid,videoids,playlist_id
0,0,I Regret Cheating On My Boyfriend But He Wont ...,★ DO YOU WANT TO GET YOUR STORY ANIMATED And F...,My Story Animated,m_xT1dmGEA4,PLb62x91woEVOzuZjzhsJTsU9fDV3fO3O8
1,1,I Stopped Eating For 2 Weeks To Get Thinner,★ DO YOU WANT TO GET YOUR STORY ANIMATED And F...,My Story Animated,znvv0qk7a-A,PLb62x91woEVOzuZjzhsJTsU9fDV3fO3O8
2,2,I Cant Stop Stealing From My Friends And Family,★ DO YOU WANT TO GET YOUR STORY ANIMATED And F...,My Story Animated,AkDbnUu0uw4,PLb62x91woEVOzuZjzhsJTsU9fDV3fO3O8
3,3,I Ran Away From Home It Was Terrifying,★ DO YOU WANT TO GET YOUR STORY ANIMATED And F...,My Story Animated,4OdmJPGG7f0,PLb62x91woEVOzuZjzhsJTsU9fDV3fO3O8
4,4,My Heart Is Broken Forever Because Of My Mother,I Hated My Mother | My heart is broken forever...,My Story Animated,VjrzlEt_uuU,PLb62x91woEVOzuZjzhsJTsU9fDV3fO3O8
...,...,...,...,...,...,...
525,645,"Crime Scene Cleanup Crew Members, What's Your ...","► Crime Scene Cleanup Crew Members, What's You...",Reddit Reveals,ymRMOwLYATM,PLVtWx-uH8EaqZEYNV0e9jmoX0sfjhV8VP
526,646,What Happens In Real Life S*x That You Never S...,► What Happens In Real Life S*x That You Never...,Reddit Reveals,n39Af7Rt8-4,PLVtWx-uH8EaqZEYNV0e9jmoX0sfjhV8VP
527,647,Teachers Reveal Their Most NSFW Story From Sch...,► Teachers Reveal Their Most NSFW Story From S...,Reddit Reveals,HgJtIHp2ySk,PLVtWx-uH8EaqZEYNV0e9jmoX0sfjhV8VP
528,648,What Are Some “Girl Secrets” Guys Don’t Know A...,► What Are Some “Girl Secrets” Guys Don’t Know...,Reddit Reveals,VjZSuG45fn0,PLVtWx-uH8EaqZEYNV0e9jmoX0sfjhV8VP


In [11]:
Complete=' '
i=0
while i<529:
    Complete +=  video_df['title'][i]+ " \n " 
    i+=1
    
Complete
    


" I Regret Cheating On My Boyfriend But He Wont Forgive Me \n I Stopped Eating For 2 Weeks To Get Thinner \n I Cant Stop Stealing From My Friends And Family \n I Ran Away From Home It Was Terrifying \n My Heart Is Broken Forever Because Of My Mother \n I Dyed my Hair Blond and Something Crazy Happened \n I Am A Very Famous PopStar, But I Feel Like A Loser \n I Am The Richest Girl At School And It's Embarrassing \n My Mom is Mentally ILL \n My Vacation Abroad Turned Into Hell \n I Became An Instagram Star Overnight \n Truth or Dare Almost Ruined My Life \n I Love My Best Friend's Boyfriend \n The Price I Made my Ex-boyfriend Pay \n I Found a Way to be Cool \n My Mom Wants Me to Get Plastic Surgery \n I Stole My Dad's Computer And I Was Not Ready For What I Saw \n 4 Emotional Videos You Must Watch \n I Envy My Crazy Pretty Stepmom \n I Ate My Rabbit \n I Envy My Extremely Rich Step-Sister \n I Found a Thief In My Room And I Outsmarted Him \n My Mom Humiliated My Black Boyfriend But Dad..

In [16]:
Complete= Complete.replace('😎','e')
Complete = Complete.replace('| This is my story',' ')
Complete= Complete.replace('(r/AskReddit | Reddit Stories)',' ')
Complete

" I Regret Cheating On My Boyfriend But He Wont Forgive Me \n I Stopped Eating For 2 Weeks To Get Thinner \n I Cant Stop Stealing From My Friends And Family \n I Ran Away From Home It Was Terrifying \n My Heart Is Broken Forever Because Of My Mother \n I Dyed my Hair Blond and Something Crazy Happened \n I Am A Very Famous PopStar, But I Feel Like A Loser \n I Am The Richest Girl At School And It's Embarrassing \n My Mom is Mentally ILL \n My Vacation Abroad Turned Into Hell \n I Became An Instagram Star Overnight \n Truth or Dare Almost Ruined My Life \n I Love My Best Friend's Boyfriend \n The Price I Made my Ex-boyfriend Pay \n I Found a Way to be Cool \n My Mom Wants Me to Get Plastic Surgery \n I Stole My Dad's Computer And I Was Not Ready For What I Saw \n 4 Emotional Videos You Must Watch \n I Envy My Crazy Pretty Stepmom \n I Ate My Rabbit \n I Envy My Extremely Rich Step-Sister \n I Found a Thief In My Room And I Outsmarted Him \n My Mom Humiliated My Black Boyfriend But Dad..

In [17]:
text_file = open("sample.txt", "w")
n = text_file.write(Complete)
text_file.close()

Este codigo de abajo solo si los videos de la lista tienen CC

In [10]:
def get_all_ccs(videos):
    base_url = 'https://www.youtube.com/watch?v='
    lang="en"
    for vid in videos:
        url = base_url + vid
        cmd = ["youtube-dl","--skip-download","--write-sub",
               "--sub-lang",lang,url]
        os.system(" ".join(cmd))

In [11]:
get_all_ccs(video_df.videoids)

KeyboardInterrupt: 

In [12]:
filenames_vtt = [os.fsdecode(file) for file in os.listdir(os.getcwd()) if os.fsdecode(file).endswith(".vtt")]
#Check file names
filenames_vtt[:2]

[]

In [ ]:
import webvtt
def convert_vtt(filenames):    
    #create an assets folder if one does not yet exist
    if os.path.isdir('{}/assets'.format(os.getcwd())) == False:
        os.makedirs('assets')
    #extract the text and times from the vtt file
    for file in filenames:
        captions = webvtt.read(file)
        text_time = pd.DataFrame()
        text_time['text'] = [caption.text for caption in captions]
        text_time['start'] = [caption.start for caption in captions]
        text_time['stop'] = [caption.end for caption in captions]
        text_time.to_csv('assets/{}.csv'.format(file[:-4]),index=False) #-4 to remove '.vtt'
        #remove files from local drive
        os.remove(file)

In [ ]:
#call the function
convert_vtt(filenames_vtt)

In [ ]:
csv_files = [os.fsdecode(file) for file in os.listdir(os.getcwd()+'/assets') if os.fsdecode(file).endswith('.csv')]
#take a look at a file name
csv_files[0]
output:
'Visualizing a column space as a plane in R3 _ Vectors and spaces _ Linear Algebra _ Khan Academy-EGNlXtjYABw.en.csv'

In [ ]:
path = 'assets/'
for filename in csv_files:
    os.rename(os.path.join(path, filename), os.path.join(path, filename.replace(' ', '')))
#to verify the results
clean_csv = [os.fsdecode(file) for file in os.listdir(os.getcwd()+'/assets')]
clean_csv[0]
output:
'VisualizingacolumnspaceasaplaneinR3_Vectorsandspaces_LinearAlgebra_KhanAcademy-EGNlXtjYABw.en.csv'

In [ ]:
#extract the text and videoid
vidText = []
csv_vidid = []

for file in clean_csv:
   df = pd.read_csv(path+file)
   text = “ “.join(df.text)
   vidText.append(text)
   csv_vidid.append(file[-18:-7])

In [ ]:
lectures_df = pd.DataFrame()
lectures_df['lecture_title'] = clean_csv
lectures_df['lecture_text'] = vidText
lectures_df['vid_id'] = csv_vidid

In [ ]:
base_df = lectures_df.set_index(‘vid_id’).join(video_df.set_index(‘videoids’)).reset_index()